In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
def load_and_split_multimodal_data(
    sensor_csv_path: str,
    image_data_dir: str,
    test_size: float = 0.1,
    val_size: float = 0.2,
    batch_size: int = 32,
    img_size: tuple = (120, 160),
    random_state: int = 42
):
    """
    Load and split both sensor CSV and image folders into multimodal tf.data.Datasets.

    Args:
        sensor_csv_path: Path to the gas sensor CSV.
        image_data_dir: Base directory of image class subfolders.
        test_size: Fraction of data reserved for testing.
        val_size: Fraction of remaining data reserved for validation.
        batch_size: Batch size for tf.data pipelines.
        img_size: Size to resize images to (height, width).
        random_state: Seed for reproducibility.

    Returns:
        train_ds, val_ds, test_ds: tf.data.Dataset yielding ((sensor, image), label).
    """
    # 1) Load sensor CSV
    sdf = pd.read_csv(sensor_csv_path)
    sdf = sdf.drop(columns=["Serial Number"], errors='ignore')
    sdf['Gas'] = sdf['Gas'].astype('category').cat.codes

    # Build image filename column
    sdf['Image_File'] = sdf['Corresponding Image Name'].astype(str) + ".png"

    # Extract sensor features and labels
    sensor_cols = [c for c in sdf.columns if c not in ['Gas', 'Corresponding Image Name', 'Image_File']]
    sensors = sdf[sensor_cols].values.astype('float32')
    labels = sdf['Gas'].values.astype('int32')

    # Normalize sensors
    scaler = StandardScaler().fit(sensors)
    sensors = scaler.transform(sensors)

    # Map image names to full paths
    base = image_data_dir
    def find_path(fname):
        for cls in os.listdir(base):
            p = os.path.join(base, cls, fname)
            if os.path.exists(p):
                return p
        return None

    paths = np.array(sdf['Image_File'].map(find_path))
    valid = ~pd.isna(paths)

    sensors = sensors[valid]
    paths   = paths[valid]
    labels  = labels[valid]

    # Split multimodal arrays
    X_temp, X_test, S_temp, S_test, y_temp, y_test = train_test_split(
        paths, sensors, labels,
        test_size=test_size, stratify=labels, random_state=random_state
    )
    val_frac = val_size / (1 - test_size)
    X_train, X_val, S_train, S_val, y_train, y_val = train_test_split(
        X_temp, S_temp, y_temp,
        test_size=val_frac, stratify=y_temp, random_state=random_state
    )

    # Define loader
    def loader(path, sens, lab):
        img = tf.io.read_file(path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, img_size)
        img = tf.cast(img, tf.float32) / 255.0
        sens = tf.cast(sens, tf.float32)
        return (sens, img), lab

    # Build tf.data datasets
    def make_ds(paths, sensors, labels, shuffle=False):
        ds = tf.data.Dataset.from_tensor_slices((paths, sensors, labels))
        ds = ds.map(loader, num_parallel_calls=tf.data.AUTOTUNE)
        if shuffle:
            ds = ds.shuffle(buffer_size=len(labels), seed=random_state)
        return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    train_ds = make_ds(X_train, S_train, y_train, shuffle=True)
    val_ds   = make_ds(X_val,   S_val,   y_val,   shuffle=False)
    test_ds  = make_ds(X_test,  S_test,  y_test,  shuffle=False)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = load_and_split_multimodal_data(
    sensor_csv_path=  "/kaggle/input/gas-dataset/zkwgkjkjn9-2/Gas Sensors Measurements/Gas_Sensors_Measurements.csv",
    image_data_dir="/kaggle/input/gas-dataset/zkwgkjkjn9-2/Thermal Camera Images")

In [ ]:
class RandomSensorDropout(tf.keras.layers.Layer):
    """
    Layer that randomly zeros individual sensor channels with a given rate during training.
    Supports proper serialization.
    """
    def __init__(self, rate=0.3, **kwargs):
        super().__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=False):
        if training and self.rate > 0.0:
            mask = tf.cast(tf.random.uniform(tf.shape(inputs)) > self.rate, inputs.dtype)
            return inputs * mask
        return inputs

    def get_config(self):
        config = super().get_config()
        config.update({"rate": self.rate})
        return config
    

# --------------- Monte Carlo Dropout Layers ---------------------    
class MCDropout(tf.keras.layers.Dropout):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference
        return super().call(inputs, training=True)
    

class MCSpatialDropout2D(tf.keras.layers.SpatialDropout2D):
    """
    Dropout that is active both at train *and* inference time,
    so we can sample N stochastic forward passes.
    """
    def call(self, inputs, training=None):
        # Force dropout even in inference
        return super().call(inputs, training=True)


In [ ]:
def build_multimodal_model(
    img_shape=(120, 160, 3),
    input_dim=7,
    sensor_dropout_rate=0.3,
    layer_dropout=0.5,
    sensor_units=32,
    img_dense=64,
    fusion_dense=64,
    output_units=4,
    lr=1e-4
):
    """
    Intermediate fusion of sensor and image branches.
    Enhanced image branch with deeper layers and normalization.
    """
    # Sensor input
    inp = tf.keras.Input(shape=(input_dim,), name="sensor_input")
    x = RandomSensorDropout(sensor_dropout_rate, name="sensor_dropout")(inp)
    x = tf.keras.layers.Dense(224, activation="relu")(x)
    x = MCDropout(0.036)(x)
    s = tf.keras.layers.Dense(240, activation="relu")(x)

    # Image input
    i_in = tf.keras.Input(shape=img_shape, name="image_input")
    x = tf.keras.layers.Conv2D(32, 3, padding="same", activation="relu" ,use_bias=False)(i_in)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(64, 3, padding="same", activation="relu", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(128, 3, padding="same", activation="relu", use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = MCSpatialDropout2D(0.2)(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(img_dense, activation="relu")(x)
    x = MCDropout(layer_dropout)(x)

    # Fusion
    fused = tf.keras.layers.Concatenate()([s, x])
    y = tf.keras.layers.Dense(fusion_dense, activation="relu")(fused)
    y = MCDropout(layer_dropout)(y)
    out = tf.keras.layers.Dense(output_units, activation="softmax", name="output")(y)

    # Model compile
    model = tf.keras.Model([inp, i_in], out, name="multimodal")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=7.07e-04),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_multimodal_model()
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_mc_sensor_model.keras', monitor='val_accuracy',
    save_best_only=True, mode='max' 
)
model.fit(train_ds , validation_data = val_ds , epochs = 100 , batch_size = 32 , callbacks = [checkpoint])


In [ ]:
best_model = tf.keras.models.load_model(
        'best_mc_sensor_model.keras',
        custom_objects={"RandomSensorDropout": RandomSensorDropout, "MCDropout": MCDropout,"MCSpatialDropout2D": MCSpatialDropout2D}
    )
def predict_mc(model, dataset, T=50):
    all_preds = []
    all_labels = []
    for sens_batch, lbl_batch in dataset:
        preds_t = [model(sens_batch).numpy() for _ in range(T)]
        preds_t = np.stack(preds_t, axis=0)  # (T, batch, classes)
        mean_preds = preds_t.mean(axis=0)    # (batch, classes)
        all_preds.append(mean_preds)
        all_labels.append(lbl_batch.numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    y_pred = np.argmax(all_preds, axis=1)
    return np.mean(y_pred == all_labels)
acc = predict_mc(best_model , test_ds)
print("testing accuracy is " ,acc)